In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import requests
import asyncio

In [2]:
data_dir = "data"
standings_dir = os.path.join(data_dir, "standings")
score_dir = os.path.join(data_dir, "scores")
#directories where we are going store our data

In [3]:
async def get_html(url, selector, sleep=5):            #async allows to perform non-blocking In/Out operations
    try:
        response = requests.get(url)
        response.raise_for_status()                    #check if the request was successful
        html_content = response.text                   #if the request is successful, it retrieves the HTML content from the response
        await asyncio.sleep(sleep)                     #asynchronously waits for the specified duration using asyncio.sleep

        return html_content
    except requests.exceptions.RequestException as e:
    
        print(f"Error occurred while fetching HTML from {url}: {e}")
        return None                                     #return None if there's an error

In [4]:
seasons = list(range(1994,2024))                      #set the seasons range for our data

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    page = requests.get(url)
    print(page.status_code)

    soup = BeautifulSoup(page.text, "html.parser")
    html_divs = soup.find_all('div', class_='filter')  

    for div in html_divs:
        soup2 = BeautifulSoup(str(div), "html.parser")  
        links = soup2.find_all("a")
        hrefs = [l.get('href') for l in links]


        standings_pages = [f"https://www.basketball-reference.com{l}" for l in hrefs]

        
        for url in standings_pages:
            save_path = os.path.join(standings_dir, url.split("/")[-1])
            if os.path.exists(save_path):
                continue

            html = await get_html(url, "#all_schedule")
            with open(save_path, "w+") as f:
                f.write(html)
                


In [6]:
for season in seasons:             #make the loop for each season
    await scrape_season(season)    #ensuring that each scrape_season call is completed before moving on to the next one
#200 shows that connection is good and season is scrapining

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [7]:
standings_files = os.listdir(standings_dir)

In [8]:
async def scrape_game(standings_files):    
    with open(standings_files, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l.get('href') for l in links]
    box_scores = [l for l in href if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(score_dir, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue        
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
standings_files = [s for s in standings_files if ".html" in s]

In [10]:
standings_files = [s for s in standings_files if any(str(year) in s for year in [2008, 2009, 2014, 2017, 1994, 1995])]

In [11]:
for f in standings_files:
    file_path = os.path.join(standings_dir, f)
        
    await scrape_game(file_path)

In [18]:
#i stopped the function above, because it's already running for 24h, so let's check how many file we got

In [13]:
score_dir = "data/scores"

In [14]:
box_score = os.listdir(score_dir)

In [15]:
len(box_score)       #check amount of files

13548

In [ ]:
# so by next step I'll parsing data